In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("movie_data.csv",encoding = "utf-8")

In [3]:
df.head(10)

,Review,Sentiment
0,"Based on an actual story, John Boorman shows t...",1
1,This is a gem. As a Film Four production - the...,1
2,"I really like this show. It has drama, romance...",1
3,This is the best 3-D experience Disney has at ...,1
4,"Of the Korean movies I've seen, only three had...",1
5,this movie is funny funny funny my favorite qu...,1
6,I'm just starting to explore the so far wonder...,1
7,There is no need for me to repeat the synopsis...,1
8,"I got this movie with my BBC ""Jane Austen Coll...",1
9,"This was a great movie, I would compare it to ...",1


In [63]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 148515312879963104
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7055162988766905774
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10450829312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17718477235924598480
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2021266661797710658
physical_device_desc: "device: XLA_GPU device"
]


In [64]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback
print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


['/device:GPU:0']

In [42]:
#Train and Test Data:
X_train = df.loc[:24999, 'Review'].values
y_train = df.loc[:24999, 'Sentiment'].values
X_test = df.loc[25000:, 'Review'].values
y_test = df.loc[25000:, 'Sentiment'].values

In [66]:
#Learning Word Embeddings, creating words closer together 
#tokenize items then embed them in a vector space
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [67]:
tokenizer = Tokenizer()
total_reviews = X_train + X_test 
tokenizer.fit_on_texts(total_reviews)



In [68]:
#pad sequences that are not the longest 
max_length = max([len(s.split()) for s in total_reviews])

In [69]:
#vocabulary size 
vocab_size = len(tokenizer.word_index) + 1
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)
X_train_padded = pad_sequences(X_train_tokens, maxlen = max_length, padding ="post")
X_test_padded = pad_sequences(X_test_tokens, maxlen = max_length, padding ="post")

In [70]:
vocab_size

125583

In [4]:
#Building the model with an embedding layer that learns the embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding

EMBEDDING_DIM = 20
#Embedding dimension is based on the computational resources at hand
print("Building Model")
#Start Sequential Model
model = Sequential()
#add first hidden layer for embedding
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length))
#Add GRU units for reducing the effect of the vanishing Gradient
model.add(GRU(units = 32, dropout = 0.2, recurrent_dropout = 0.2))
#Dense tells me what type of movie sentiment it has between 0 and 1, set a threshold
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = "binary_crossentropy", optimizer = "adam",metrics = ["accuracy"])

Building Model


NameError: name 'vocab_size' is not defined

In [73]:
print("Train")
model.fit(X_train_padded,y_train,batch_size=1,epochs = 25,verbose = 2)

Train
Train on 25000 samples
Epoch 1/25


KeyboardInterrupt: 

In [6]:
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

review_lines = list()
lines = df["Review"].values.tolist()

for line in lines:
    tokens = word_tokenize(line)
    #lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from words
    table = str.maketrans('','',string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    #stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    review_lines.append(words)

[nltk_data] Downloading package punkt to /home/adumarathe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/adumarathe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
len(review_lines)

50000

In [ ]:
import gensim